# Lesson 1: Advanced RAG Pipeline

In [32]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [33]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [34]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: f63c1003-937c-4325-b539-527a20b7cd8a
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [35]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [36]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [37]:
query_engine = index.as_query_engine()

In [38]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

When finding projects to build your experience, there are several steps you can take. First, you can join existing projects by asking to join someone else's project if they have an idea. Additionally, you can develop a side hustle or personal project that may or may not develop into something bigger. It's important to choose projects that will help you grow technically, but are also challenging enough to stretch your skills without being too difficult. Having good teammates or people to discuss things with is also important, as you can learn a lot from the people around you. Finally, consider if the project can be a stepping stone to larger projects in terms of technical complexity and business impact.


## Evaluation setup using TruLens

In [39]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [40]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [41]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [42]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [43]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [44]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [45]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [46]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fe8327418bf727b51a190b0a1f18e8ea,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_fe8327418bf727b51a1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T09:18:05.194478"", ""...",2023-12-02T09:18:09.226558,1.0,0.9,1.000,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",4,2180,0.003351
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_db20832a40bc759ca59ec658f298de64,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_db20832a40bc759ca59...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T09:18:09.415025"", ""...",2023-12-02T09:18:11.642646,1.0,0.0,1.000,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,2,1712,0.002612
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5da84cb5b6d142ac24357931eb382e1b,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_5da84cb5b6d142ac243...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T09:18:11.830380"", ""...",2023-12-02T09:18:14.620496,0.9,0.0,0.250,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,2,1699,0.002586
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a0dc568ac7fdc39c367cf08192d83222,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_a0dc568ac7fdc39c367...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T09:18:14.819664"", ""...",2023-12-02T09:18:17.459392,0.9,0.6,0.750,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,2,1676,0.002554
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_44a60417f409af801a14d8bac6677172,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_44a60417f409af801a1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-02T09:18:17.654731"", ""...",2023-12-02T09:18:19.973695,1.0,0.0,0.625,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,2,1669,0.002541


In [47]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-31-15-12p19319.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [48]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [49]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [50]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [51]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, it is important to first identify and scope the project. Consider your career goals and choose a project that complements them. Ensure that the project is responsible, ethical, and beneficial to people. As you progress in your career, aim for projects that grow in scope, complexity, and impact over time. Building a portfolio of projects that shows skill progression can also be helpful. Additionally, consider using a simple framework for starting your AI job search and utilize informational interviews to find the right job in the AI field.


In [52]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [53]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7134105db0 is calling an instrumented method <function BaseQueryEngine.query at 0x7f72b3f3aef0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7134105db0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f72af422b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x7f7134104e20 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f72b3f3a290>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obj

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71341060e0 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7134105db0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f72af422b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.


How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively as a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71341060e0 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7134105db0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f72af422b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, collaborating with stakeholders who lack expertise in AI, and struggling with technical challenges while reading research papers or tuning neural network hyperparameters.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71341060e0 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f7134105db0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f72af422b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.


Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The author mentions that they still find many research papers challenging to read and have made mistakes while working with neural networks. They also assure the reader that everyone who has published a seminal AI paper has struggled with similar technical challenges at some point. Therefore, it is clear that finding parts of AI challenging is a common experience.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71341060e0 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.


What is the right AI job for me?
The right AI job for you can be found by following the steps outlined in Chapter 9: Finding the Right AI Job for You. This chapter provides guidance on how to identify and pursue AI job opportunities that align with your skills, interests, and career goals. Additionally, Chapter 8: Using Informational Interviews to Find the Right Job offers valuable insights on how to gather information and make informed decisions about potential AI job opportunities. By leveraging the information and frameworks provided in these chapters, you can find the AI job that is the best fit for you.


In [54]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.98,0.875,0.277778,6.272727,0.000852


In [55]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-31-15-12p19319.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [56]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [57]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [58]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: d4292400-5f58-40f4-93d1-189b70ae8ed9.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: f30ae8f8-1855-4dd8-b067-ce0d3438213c.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression will demonstrate your growth and development over time. Additionally, effective communication is crucial. You should be able to explain your thinking and articulate the value of your work to others. This will help others see the potential in your projects and trust you with the necessary resources for larger endeavors.


In [59]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [60]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f71342d7a90 is calling an instrumented method <function BaseQueryEngine.query at 0x7f72b3f3aef0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f71342d7a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f72af422b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f7134231600 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f72b3f3a290>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: 0256203c-507f-497d-9780-2ea81c4740af.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 939587f0-b677-4b74-ba6c-8c7425d05b94.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71342d6b30 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f72b397dea0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71342d6b30 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7f6feb227ec0 is calling an instrumented method <function LLMPredictor.predict at 0x7f72bf7040d0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a community. Additionally, collaborating with others and being able to influence and be influenced by others is critical for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71342d6b30 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f71342d7a90 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f72af422b90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f7134233a90) using this function.


How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to collaborate with others, influence their teammates, and be influenced by them. This is important because when tackling large projects, working in teams is more effective than working individually. Additionally, teamwork helps in developing interpersonal and communication skills, which are crucial in the field of AI.
> Merging 3 nodes into parent node.
> Parent node id: 19c0865e-bfd7-47a2-bcac-cd5ea2d5a6a0.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 1 nodes into parent node.
> Parent node id: 0fed46d5-00e0-4f15-bef1-fd19324872f9.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f71342d6b30 is calling an instrumented method <function Refine.get_response at 0x7f72b2790ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f72aef357e0) using this function.
Error calling wrapped function predict.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/trulens_eval/instruments.py", line 738, in wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/usr/local/lib/python3.10/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 377, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/usr/local/lib/python3.10/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 303, in track_all_costs
    return Endpoint._track_costs(thunk, with_endpoints=endpoints)
  File "/usr/local/lib/python3.10/site-packages

KeyboardInterrupt: 

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()